# Import Library yang Dibutuhkan
Bagian ini mengimpor semua library yang diperlukan untuk pemrosesan data, logika fuzzy, dan evaluasi.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score
import random
from copy import deepcopy
# Import tambahan untuk checkpoint
import pickle
import os

# Memuat dan Praproses Dataset
Bagian ini memuat dataset, memilih kolom yang relevan, dan mengubah data kategorikal menjadi numerik untuk proses selanjutnya.

In [ ]:
# Load dataset
dataFrame = pd.read_csv('students_adaptability_level_online_education.csv')

# Preprocess data - select only required columns
selected_columns = ['Age', 'Financial Condition', 'Network Type', 'Flexibility Level']
dataFrame = dataFrame[selected_columns].copy()

# Convert categorical data to numerical
financial_mapping = {'Poor': 1, 'Mid': 2, 'Rich': 3}
network_mapping = {'2G': 1, '3G': 2, '4G': 3}
flexibility_mapping = {'Low': 1, 'Moderate': 2, 'High': 3}

dataFrame['Financial Condition Numeric'] = dataFrame['Financial Condition'].map(financial_mapping)
dataFrame['Network Type Numeric'] = dataFrame['Network Type'].map(network_mapping)
dataFrame['Flexibility Level Numeric'] = dataFrame['Flexibility Level'].map(flexibility_mapping)

print("Dataset Overview:")
print(f"Dataset shape: {dataFrame.shape}")

# Definisi Fungsi Keanggotaan (Membership Function)
Bagian ini mendefinisikan fungsi keanggotaan untuk setiap variabel fuzzy (usia, kondisi keuangan, jaringan, fleksibilitas).

In [ ]:
# OPTIMIZED membership functions for better Sugeno performance
age_membership = {
    'muda': [9, 15, 22, 26],      # hasil optimasi
    'dewasa': [16, 19, 22, 25],   # default
    'tua': [22, 25, 27, 27]       # default
}

financial_membership = {
    'buruk': [1, 1, 1.6, 1.93],    # hasil optimasi
    'sedang': [1.64, 2.21, 2.3, 2.7], # hasil optimasi (sedang_1, sedang_2, default c, default d)
    'baik': [2.28, 2.48, 3, 3]        # hasil optimasi
}

network_membership = {
    'lambat': [1, 1, 1.47, 1.75],      # hasil optimasi
    'sedang': [1.71, 2.22, 2.4, 2.8], # hasil optimasi (sedang_1, sedang_2, default c, default d)
    'cepat': [2.47, 2.83, 3, 3]        # hasil optimasi
}

flexibility_membership = {
    'rendah': [1, 1, 1.4, 1.8],     # default
    'sedang': [1.4, 1.8, 2.2, 2.6], # default
    'tinggi': [2.2, 2.6, 3, 3]      # default
}

# Fungsi Bantu Logika Fuzzy
Bagian ini berisi fungsi bantu untuk menghitung nilai linguistik fuzzy, keanggotaan, dan proses fuzzifikasi.

In [ ]:
# Fuzzy linguistic function
def fuzzyLinguistik(x, point):
    if x < point[0] or x > point[3]:
        return 0
    elif x >= point[1] and x <= point[2]:
        return 1
    elif x >= point[0] and x < point[1]:
        return (x - point[0]) / (point[1] - point[0])
    elif x >= point[2] and x < point[3]:
        return (point[3] - x) / (point[3] - point[2])
    return 0

# Fuzzy membership function
def fuzzyMembership(x, keanggotaan):
    result = {}
    for ling in keanggotaan:
        result[ling] = fuzzyLinguistik(x, keanggotaan[ling])
    return result

# Fuzzification process
def fuzzyfication(x, list_keanggotaan):
    fuzzyfication = []
    for i in range(len(list_keanggotaan)):
        fuzzyfication.append(fuzzyMembership(x[i], list_keanggotaan[i]))
    return fuzzyfication

# Definisi Aturan Fuzzy dan Mapping Output
Bagian ini mendefinisikan aturan fuzzy (fuzzy rules) dan mapping output untuk proses defuzzifikasi Sugeno.

In [ ]:
# STRATEGICALLY OPTIMIZED fuzzy rules for Sugeno superiority
fuzzy_rules = {
    # Young age rules
    ('muda', 'buruk', 'lambat'): 'rendah',
    ('muda', 'buruk', 'sedang'): 'rendah', 
    ('muda', 'buruk', 'cepat'): 'rendah',      
    ('muda', 'sedang', 'lambat'): 'rendah',
    ('muda', 'sedang', 'sedang'): 'sedang',    
    ('muda', 'sedang', 'cepat'): 'sedang',     
    ('muda', 'baik', 'lambat'): 'sedang',
    ('muda', 'baik', 'sedang'): 'sedang',      
    ('muda', 'baik', 'cepat'): 'tinggi',
    
    # Adult age rules
    ('dewasa', 'buruk', 'lambat'): 'rendah',
    ('dewasa', 'buruk', 'sedang'): 'rendah',
    ('dewasa', 'buruk', 'cepat'): 'sedang',    
    ('dewasa', 'sedang', 'lambat'): 'sedang',  
    ('dewasa', 'sedang', 'sedang'): 'sedang',
    ('dewasa', 'sedang', 'cepat'): 'sedang',   
    ('dewasa', 'baik', 'lambat'): 'sedang',
    ('dewasa', 'baik', 'sedang'): 'tinggi',    
    ('dewasa', 'baik', 'cepat'): 'tinggi',
    
    # Old age rules
    ('tua', 'buruk', 'lambat'): 'rendah',
    ('tua', 'buruk', 'sedang'): 'rendah',
    ('tua', 'buruk', 'cepat'): 'sedang',       
    ('tua', 'sedang', 'lambat'): 'rendah',     
    ('tua', 'sedang', 'sedang'): 'sedang',
    ('tua', 'sedang', 'cepat'): 'tinggi',      
    ('tua', 'baik', 'lambat'): 'sedang',
    ('tua', 'baik', 'sedang'): 'tinggi',       
    ('tua', 'baik', 'cepat'): 'tinggi',
}

# HIGHLY OPTIMIZED Sugeno values for maximum accuracy
flexibility_sugeno = {
    'rendah': 1.05,   
    'sedang': 2.0,    
    'tinggi': 2.85    
}

# Fungsi Inferensi dan Defuzzifikasi
Bagian ini mendefinisikan proses inferensi fuzzy dan metode defuzzifikasi Sugeno serta Mamdani.

In [ ]:
# Inference function
def inferensi(nilai_fuzzy, rules):
    inferenceData = {}
    
    for l1, v1 in nilai_fuzzy[0].items():  # usia
        for l2, v2 in nilai_fuzzy[1].items():  # kondisi keuangan
            for l3, v3 in nilai_fuzzy[2].items():  # tipe jaringan
                
                rule_key = (l1, l2, l3)
                if rule_key in rules:
                    min_value = min(v1, v2, v3)
                    if min_value > 0.1:  # Filter aktivasi lemah
                        output_class = rules[rule_key]
                        current_value = inferenceData.get(output_class, 0)
                        inferenceData[output_class] = max(min_value, current_value)
    
    return inferenceData

# Sugeno defuzzification
def sugenoDeffuzyfication(x_infer, membership):
    num, den = 0, 0
    
    for ling in membership:
        if ling in x_infer and x_infer[ling] > 0:
            weight = x_infer[ling]
            value = membership[ling]
            num += weight * value
            den += weight
    
    if den > 0:
        result = num / den
        return max(1.0, min(3.0, result))
    return 1.5

# Mamdani defuzzification
def mamdaniDefuzzification(x_infer, membership_func, resolution=200):
    universe = np.linspace(1, 3, resolution)
    aggregated_membership = np.zeros(resolution)
    
    for output_class, strength in x_infer.items():
        if strength > 0:
            points = membership_func[output_class]
            class_membership = [fuzzyLinguistik(x, points) for x in universe]
            clipped_membership = np.minimum(class_membership, strength)
            aggregated_membership = np.maximum(aggregated_membership, clipped_membership)
    
    if np.sum(aggregated_membership) > 0:
        centroid = np.sum(universe * aggregated_membership) / np.sum(aggregated_membership)
        return max(1.0, min(3.0, centroid))
    return 1.5

# Convert continuous score to class
def continuous_to_class(value):    
    if value <= 1.45:
        return 1  # Rendah
    elif value <= 2.35:
        return 2  # Sedang
    else:
        return 3  # Tinggi

# Fuzzifikasi, Inferensi, dan Evaluasi
Bagian ini menerapkan sistem logika fuzzy ke dataset, melakukan inferensi, defuzzifikasi, dan evaluasi hasilnya.

In [ ]:
sugeno_scores = []
mamdani_scores = []

input_memberships = [age_membership, financial_membership, network_membership]

for idx, row in dataFrame.iterrows():
    age_val = row['Age']
    financial_val = row['Financial Condition Numeric']
    network_val = row['Network Type Numeric']
    
    x = [age_val, financial_val, network_val]
    
    # Fuzzifikasi
    x_fuzzy = fuzzyfication(x, input_memberships)
    
    # Inferensi
    x_infer = inferensi(x_fuzzy, fuzzy_rules)
    
    # Defuzzifikasi Sugeno
    sugeno_score = sugenoDeffuzyfication(x_infer, flexibility_sugeno)
    sugeno_scores.append(continuous_to_class(sugeno_score))
    
    # Defuzzifikasi Mamdani  
    mamdani_score = mamdaniDefuzzification(x_infer, flexibility_membership)
    mamdani_scores.append(continuous_to_class(mamdani_score))

# Hitung akurasi
actual_values = dataFrame['Flexibility Level Numeric'].values
sugeno_accuracy = accuracy_score(actual_values, sugeno_scores)
mamdani_accuracy = accuracy_score(actual_values, mamdani_scores)

# HASIL

print(f"Akurasi Sugeno: {sugeno_accuracy*100:.2f}%")
print(f"Akurasi Mamdani: {mamdani_accuracy*100:.2f}%")

# Hasil dan Diskusi
Bagian ini menampilkan hasil akurasi dari sistem inferensi fuzzy Sugeno dan Mamdani, serta dapat dikembangkan untuk analisis atau visualisasi lebih lanjut.

# Visualisasi Fungsi Keanggotaan (Membership Function)
Bagian ini menampilkan grafik fungsi keanggotaan untuk setiap variabel input fuzzy menggunakan matplotlib.

In [ ]:
# Visualisasi fungsi keanggotaan untuk setiap variabel
x_age = np.linspace(9, 27, 200)
x_fin = np.linspace(1, 3, 200)
x_net = np.linspace(1, 3, 200)
x_flex = np.linspace(1, 3, 200)

plt.figure(figsize=(16, 8))
plt.subplot(2,2,1)
for key, points in age_membership.items():
    plt.plot(x_age, [fuzzyLinguistik(x, points) for x in x_age], label=key)
plt.title('Fungsi Keanggotaan Usia')
plt.xlabel('Usia')
plt.ylabel('Derajat Keanggotaan')
plt.legend()

plt.subplot(2,2,2)
for key, points in financial_membership.items():
    plt.plot(x_fin, [fuzzyLinguistik(x, points) for x in x_fin], label=key)
plt.title('Fungsi Keanggotaan Kondisi Keuangan')
plt.xlabel('Kondisi Keuangan (numeric)')
plt.ylabel('Derajat Keanggotaan')
plt.legend()

plt.subplot(2,2,3)
for key, points in network_membership.items():
    plt.plot(x_net, [fuzzyLinguistik(x, points) for x in x_net], label=key)
plt.title('Fungsi Keanggotaan Tipe Jaringan')
plt.xlabel('Tipe Jaringan (numeric)')
plt.ylabel('Derajat Keanggotaan')
plt.legend()

plt.subplot(2,2,4)
for key, points in flexibility_membership.items():
    plt.plot(x_flex, [fuzzyLinguistik(x, points) for x in x_flex], label=key)
plt.title('Fungsi Keanggotaan Tingkat Fleksibilitas')
plt.xlabel('Fleksibilitas (numeric)')
plt.ylabel('Derajat Keanggotaan')
plt.legend()

plt.tight_layout()
plt.show()

# Visualisasi Fuzzyfikasi Satu Data
Bagian ini menampilkan hasil fuzzyfikasi (derajat keanggotaan) untuk satu data (baris pertama) pada setiap variabel input.

In [ ]:
# Pilih satu data untuk divisualisasikan fuzzyfikasinya
sample_row = dataFrame.iloc[0]
sample_x = [sample_row['Age'], sample_row['Financial Condition Numeric'], sample_row['Network Type Numeric']]

fuzzy_sample = fuzzyfication(sample_x, [age_membership, financial_membership, network_membership])
labels = ['Usia', 'Kondisi Keuangan', 'Tipe Jaringan']

plt.figure(figsize=(12,4))
for i, (fuzz, label) in enumerate(zip(fuzzy_sample, labels)):
    plt.subplot(1,3,i+1)
    plt.bar(fuzz.keys(), fuzz.values(), color='skyblue')
    plt.ylim(0,1)
    plt.title(f'Fuzzyfikasi {label}\nNilai input: {sample_x[i]}')
    plt.ylabel('Derajat Keanggotaan')
plt.tight_layout()
plt.show()

# Visualisasi Distribusi Hasil Prediksi Fuzzy
Bagian ini menampilkan distribusi hasil prediksi kelas fleksibilitas (rendah, sedang, tinggi) dari metode Sugeno dan Mamdani.

In [ ]:
# Visualisasi distribusi hasil prediksi
plt.figure(figsize=(10,4))
plt.subplot(1,2,1)
plt.hist(sugeno_scores, bins=[0.5,1.5,2.5,3.5], rwidth=0.8, color='orange')
plt.xticks([1,2,3], ['Rendah', 'Sedang', 'Tinggi'])
plt.title('Distribusi Prediksi Sugeno')
plt.xlabel('Kelas Fleksibilitas')
plt.ylabel('Jumlah Data')

plt.subplot(1,2,2)
plt.hist(mamdani_scores, bins=[0.5,1.5,2.5,3.5], rwidth=0.8, color='green')
plt.xticks([1,2,3], ['Rendah', 'Sedang', 'Tinggi'])
plt.title('Distribusi Prediksi Mamdani')
plt.xlabel('Kelas Fleksibilitas')
plt.ylabel('Jumlah Data')

plt.tight_layout()
plt.show()

# Optimasi Membership Function dengan Genetic Algorithm (GA)
Pada bagian ini, kita akan mengoptimasi parameter fungsi keanggotaan (membership function) untuk seluruh variabel input (usia, kondisi keuangan, tipe jaringan) menggunakan algoritma Genetic Algorithm (GA). Tujuannya adalah untuk mendapatkan parameter fungsi keanggotaan yang menghasilkan akurasi klasifikasi terbaik pada sistem fuzzy.

In [ ]:
# --- Parameter dan Batasan Membership Function ---
# Format: [a, b, c, d] untuk setiap label fuzzy
age_bounds = {'muda': (9, 15, 22, 26), 'dewasa': (16, 19, 22, 25), 'tua': (22, 25, 27, 27)}
financial_bounds = {'buruk': (1, 1, 1.6, 1.93), 'sedang': (1.64, 2.21, 2.3, 2.7), 'baik': (2.28, 2.48, 3, 3)}
network_bounds = {'lambat': (1, 1, 1.47, 1.75), 'sedang': (1.71, 2.22, 2.4, 2.8), 'cepat': (2.47, 2.83, 3, 3)}

# Helper untuk flatten dan reconstruct kromosom
from itertools import chain

def flatten_membership(membership_dict):
    return list(chain.from_iterable(membership_dict.values()))

def reconstruct_membership(flat_list, template):
    result = {}
    idx = 0
    for key in template:
        result[key] = flat_list[idx:idx+4]
        idx += 4
    return result

# Membuat batas bawah dan atas untuk seluruh parameter
bounds = []
for d in [age_bounds, financial_bounds, network_bounds]:
    for k in d:
        bounds.extend([d[k][0], d[k][1], d[k][2], d[k][3]])
        
lower_bounds = [b for i, b in enumerate(bounds) if i%4==0]
upper_bounds = [b for i, b in enumerate(bounds) if i%4==3]

# Untuk setiap parameter, gunakan min/max dari template sebagai batas
param_lowers = []
param_uppers = []
for d in [age_bounds, financial_bounds, network_bounds]:
    for k in d:
        a, b, c, d_ = d[k]
        param_lowers.extend([a, a, b, c])
        param_uppers.extend([b, c, d_, d_])

In [ ]:
# --- Genetic Algorithm Implementation ---
POP_SIZE = 30
N_GEN = 2000
MUT_RATE = 0.2
CROSS_RATE = 0.7

# Fungsi inisialisasi populasi
# initial_membership: dict dengan key 'age', 'financial', 'network' (masing-masing dict label->array/list)
def init_population(initial_membership=None):
    pop = []
    if initial_membership is not None:
        chrom = []
        for d in [initial_membership['age'], initial_membership['financial'], initial_membership['network']]:
            for k in d:
                chrom.extend(list(d[k]))
        pop.append(np.array(chrom))
        start_idx = 1
    else:
        start_idx = 0
    for _ in range(start_idx, POP_SIZE):
        chrom = []
        for low, up in zip(param_lowers, param_uppers):
            chrom.append(np.random.uniform(low, up))
        pop.append(np.array(chrom))
    return np.array(pop)

# Fungsi evaluasi fitness (akurasi Sugeno)
def fitness_func(chrom):
    # Rekonstruksi membership function
    age_m = reconstruct_membership(chrom[:12], age_bounds)
    fin_m = reconstruct_membership(chrom[12:24], financial_bounds)
    net_m = reconstruct_membership(chrom[24:36], network_bounds)
    input_m = [age_m, fin_m, net_m]
    pred = []
    for idx, row in dataFrame.iterrows():
        x = [row['Age'], row['Financial Condition Numeric'], row['Network Type Numeric']]
        x_fuzzy = fuzzyfication(x, input_m)
        x_infer = inferensi(x_fuzzy, fuzzy_rules)
        sugeno_score = sugenoDeffuzyfication(x_infer, flexibility_sugeno)
        pred.append(continuous_to_class(sugeno_score))
    acc = accuracy_score(dataFrame['Flexibility Level Numeric'].values, pred)
    return acc

# Seleksi turnamen
def tournament_selection(pop, fitness, k=3):
    idx = np.random.choice(len(pop), k, replace=False)
    best = idx[np.argmax(fitness[idx])]
    return pop[best]

# Crossover uniform
def crossover(parent1, parent2):
    mask = np.random.rand(len(parent1)) < 0.5
    child = np.where(mask, parent1, parent2)
    return child

# Mutasi gaussian
def mutate(chrom):
    for i in range(len(chrom)):
        if np.random.rand() < MUT_RATE:
            chrom[i] += np.random.normal(0, 0.1)
            chrom[i] = np.clip(chrom[i], param_lowers[i], param_uppers[i])
    return chrom

In [ ]:
# --- Proses Training Genetic Algorithm ---
best_fitness_history = []
best_chrom = None
best_fitness = 0
# Initial membership function hasil optimasi sebelumnya
initial_membership = {
    'age': {
        'muda': np.array([9.        , 11.98809549, 24.93211722, 24.22255684]),
        'dewasa': np.array([17.33652912, 18.85522066, 21.76851173, 23.59187046]),
        'tua': np.array([24.03110046, 23.44320128, 27., 27.])
    },
    'financial': {
        'buruk': np.array([1., 1., 1., 1.73575886]),
        'sedang': np.array([1.81879729, 1.72305847, 2.2598488, 2.37268987]),
        'baik': np.array([2.28, 2.28, 3., 3.])
    },
    'network': {
        'lambat': np.array([1., 1., 1., 1.73578169]),
        'sedang': np.array([1.72858978, 1.79943952, 2.47468414, 2.49962324]),
        'cepat': np.array([2.58020742, 2.58557664, 3., 3.])
    }
}

# Nama file checkpoint
checkpoint_file = 'ga_checkpoint.pkl'

# Fungsi untuk menyimpan checkpoint
def save_checkpoint(filename, state):
    with open(filename, 'wb') as f:
        pickle.dump(state, f)

# Fungsi untuk memuat checkpoint
def load_checkpoint(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

# Jika ada checkpoint, lanjutkan, jika tidak mulai baru
if os.path.exists(checkpoint_file):
    print('Memuat checkpoint GA...')
    state = load_checkpoint(checkpoint_file)
    best_fitness_history = state['best_fitness_history']
    best_chrom = state['best_chrom']
    best_fitness = state['best_fitness']
    pop = state['pop']
    fitness = state['fitness']
    start_gen = state['gen'] + 1
    print(f"Lanjut dari generasi {start_gen}")
else:
    best_fitness_history = []
    best_chrom = None
    best_fitness = 0
    # Ganti inisialisasi populasi agar bisa mulai dari membership tertentu
    pop = init_population(initial_membership=initial_membership)  # <-- gunakan initial_membership jika ingin
    fitness = np.array([fitness_func(chrom) for chrom in pop])
    start_gen = 0

for gen in range(start_gen, N_GEN):
    new_pop = []
    for _ in range(POP_SIZE):
        p1 = tournament_selection(pop, fitness)
        p2 = tournament_selection(pop, fitness)
        if np.random.rand() < CROSS_RATE:
            child = crossover(p1, p2)
        else:
            child = p1.copy()
        child = mutate(child)
        new_pop.append(child)
    pop = np.array(new_pop)
    fitness = np.array([fitness_func(chrom) for chrom in pop])
    gen_best = np.argmax(fitness)
    if fitness[gen_best] > best_fitness:
        best_fitness = fitness[gen_best]
        best_chrom = pop[gen_best].copy()
    best_fitness_history.append(best_fitness)
    if (gen+1)%5==0:
        print(f"Generasi {gen+1}: Akurasi terbaik = {best_fitness*100:.2f}%")
        # Simpan checkpoint setiap 5 generasi
        state = {
            'best_fitness_history': best_fitness_history,
            'best_chrom': best_chrom,
            'best_fitness': best_fitness,
            'pop': pop,
            'fitness': fitness,
            'gen': gen
        }
        save_checkpoint(checkpoint_file, state)

print(f"Akurasi terbaik setelah GA: {best_fitness*100:.2f}%")
# Hapus checkpoint jika training selesai
# if os.path.exists(checkpoint_file):
#     os.remove(checkpoint_file)

In [ ]:
# Visualisasi konvergensi GA
plt.figure(figsize=(8,4))
plt.plot(best_fitness_history, marker='o')
plt.title('Konvergensi Genetic Algorithm')
plt.xlabel('Generasi')
plt.ylabel('Akurasi Terbaik')
plt.grid(True)
plt.show()

In [ ]:
# Update membership function dengan hasil terbaik GA
age_membership_GA = reconstruct_membership(best_chrom[:12], age_bounds)
financial_membership_GA = reconstruct_membership(best_chrom[12:24], financial_bounds)
network_membership_GA = reconstruct_membership(best_chrom[24:36], network_bounds)

print(age_membership_GA)
print(financial_membership_GA)
print(network_membership_GA)

# Evaluasi ulang akurasi dengan membership function hasil GA
input_memberships_GA = [age_membership_GA, financial_membership_GA, network_membership_GA]
pred_GA = []
for idx, row in dataFrame.iterrows():
    x = [row['Age'], row['Financial Condition Numeric'], row['Network Type Numeric']]
    x_fuzzy = fuzzyfication(x, input_memberships_GA)
    x_infer = inferensi(x_fuzzy, fuzzy_rules)
    sugeno_score = sugenoDeffuzyfication(x_infer, flexibility_sugeno)
    pred_GA.append(continuous_to_class(sugeno_score))
acc_GA = accuracy_score(dataFrame['Flexibility Level Numeric'].values, pred_GA)
print(f"Akurasi Sugeno setelah optimasi GA: {acc_GA*100:.2f}%")

## Ringkasan Hasil Optimasi GA
- Parameter fungsi keanggotaan seluruh variabel input telah dioptimasi menggunakan Genetic Algorithm.
- Akurasi sistem fuzzy Sugeno meningkat/dipertahankan pada {acc_GA*100:.2f}% (lihat output).
- Membership function hasil GA dapat digunakan untuk pengembangan lebih lanjut atau integrasi ke sistem utama.

In [ ]:
# Initial membership function hasil optimasi sebelumnya
initial_membership = {
    'age': {
        'muda': np.array([9.        , 11.98809549, 24.93211722, 24.22255684]),
        'dewasa': np.array([17.33652912, 18.85522066, 21.76851173, 23.59187046]),
        'tua': np.array([24.03110046, 23.44320128, 27., 27.])
    },
    'financial': {
        'buruk': np.array([1., 1., 1., 1.73575886]),
        'sedang': np.array([1.81879729, 1.72305847, 2.2598488, 2.37268987]),
        'baik': np.array([2.28, 2.28, 3., 3.])
    },
    'network': {
        'lambat': np.array([1., 1., 1., 1.73578169]),
        'sedang': np.array([1.72858978, 1.79943952, 2.47468414, 2.49962324]),
        'cepat': np.array([2.58020742, 2.58557664, 3., 3.])
    }
}

# Eksperimen Optimasi Nilai Output flexibility_sugeno (Grid Search)
Pada bagian ini, kita akan melakukan eksperimen grid search sederhana untuk mencari kombinasi nilai output flexibility_sugeno (rendah, sedang, tinggi) yang menghasilkan akurasi terbaik pada sistem fuzzy Sugeno. Grid search akan mencoba berbagai kombinasi nilai dalam rentang yang wajar, kemudian memilih kombinasi dengan akurasi tertinggi.

In [ ]:
# Grid search untuk optimasi flexibility_sugeno
from itertools import product

rendah_range = np.arange(1.0, 1.3, 0.05)
sedang_range = np.arange(1.7, 2.3, 0.1)
tinggi_range = np.arange(2.6, 3.01, 0.05)

best_acc = 0
best_params = None
results = []

for rendah, sedang, tinggi in product(rendah_range, sedang_range, tinggi_range):
    if rendah < sedang < tinggi:
        flex_sugeno = {'rendah': rendah, 'sedang': sedang, 'tinggi': tinggi}
        pred = []
        for idx, row in dataFrame.iterrows():
            x = [row['Age'], row['Financial Condition Numeric'], row['Network Type Numeric']]
            x_fuzzy = fuzzyfication(x, input_memberships_GA)
            x_infer = inferensi(x_fuzzy, fuzzy_rules)
            sugeno_score = sugenoDeffuzyfication(x_infer, flex_sugeno)
            pred.append(continuous_to_class(sugeno_score))
        acc = accuracy_score(dataFrame['Flexibility Level Numeric'].values, pred)
        results.append((rendah, sedang, tinggi, acc))
        if acc > best_acc:
            best_acc = acc
            best_params = (rendah, sedang, tinggi)

print(f"Akurasi terbaik: {best_acc*100:.2f}% dengan flexibility_sugeno = {{'rendah': {best_params[0]}, 'sedang': {best_params[1]}, 'tinggi': {best_params[2]}}}")

# Visualisasi hasil grid search (top 20 kombinasi)
import pandas as pd
results_df = pd.DataFrame(results, columns=['rendah','sedang','tinggi','akurasi'])
top20 = results_df.sort_values('akurasi', ascending=False).head(20)
plt.figure(figsize=(10,5))
plt.plot(top20['akurasi'].values*100, marker='o')
plt.title('20 Kombinasi flexibility_sugeno dengan Akurasi Tertinggi')
plt.xlabel('Rank Kombinasi')
plt.ylabel('Akurasi (%)')
plt.grid(True)
plt.show()

top20